# FAANG Stock Analysis and Predictor

In this notebook, we are taking a look at the five most popular and best performing American technology companies: Facebook, Amazon, Apple, Netflix, and Google. In the notebook, there will be an analysis on the stock performance in the year 2020, as well as a prediction algorithm (using machine learning) that will be run on each of the five stocks to ulimately provide a complete analysis of all five stocks.

### Table Of Contents

* [Setup](#setup)


## Setup <a class="anchor" id="setup"></a>

In [3]:
import math
from datetime import date, timedelta
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import pandas_datareader as web
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

In [4]:
# Get the data from all of the FAANG stocks
five_years_ago = date.today() - timedelta(days=5*365)
fb_data = web.DataReader('FB', data_source='yahoo', start=five_years_ago, end=date.today())
amzn_data = web.DataReader('AMZN', data_source='yahoo', start=five_years_ago, end=date.today())
aapl_data = web.DataReader('AAPL', data_source='yahoo', start=five_years_ago, end=date.today())
nflx_data = web.DataReader('NFLX', data_source='yahoo', start=five_years_ago, end=date.today())
goog_data = web.DataReader('GOOG', data_source='yahoo', start=five_years_ago, end=date.today())

In [5]:
# Put all of the adj. closing prices into one dataframe
tickers = ['FB', 'AMZN', 'AAPL', 'NFLX', 'GOOG']
faang_data = pd.DataFrame(data=list, index=fb_data.index, columns=tickers)
faang_data['FB'] = fb_data['Adj Close']
faang_data['AMZN'] = amzn_data['Adj Close']
faang_data['AAPL'] = aapl_data['Adj Close']
faang_data['NFLX'] = nflx_data['Adj Close']
faang_data['GOOG'] = goog_data['Adj Close']

In [6]:
# Show the data
faang_data

,FB,AMZN,AAPL,NFLX,GOOG
Date,,,,,
2015-10-22,99.669998,563.909973,26.683798,97.320000,651.789978
2015-10-23,102.190002,599.030029,27.510878,100.040001,702.000000
2015-10-26,103.769997,608.609985,26.632969,103.040001,712.780029
2015-10-27,103.699997,611.010010,26.464319,103.070000,708.489990
2015-10-28,104.199997,617.099976,27.554773,105.800003,712.950012
...,...,...,...,...,...
2020-10-14,271.820007,3363.709961,121.190002,541.450012,1568.079956
2020-10-15,266.720001,3338.649902,120.709999,541.940002,1559.130005
2020-10-16,265.929993,3272.709961,119.019997,530.789978,1573.010010


## Technical Indicators Analysis
1. Simple Moving Averages (SMA)
2. Exponential Moving Averages (EMA)
3. Relative Strength Index (RSI)
4. Moving Average Convergence/Divergence (MACD)
5. Stochastic Oscillator
6. Bollinger Bands


In [7]:
# Function to get the SMA

def get_SMA(ticker, period):
    SMA = pd.DataFrame()
    SMA['SMA'] = faang_data[ticker].rolling(window=period).mean()
    return SMA

In [8]:
# Function to get the EMA

def get_EMA(ticker, period):
    EMA = pd.DataFrame()
    EMA['EMA'] = faang_data[ticker].ewm(span=period).mean()
    
    return EMA

In [50]:
# Function to get the RSI

def get_RSI(ticker):
    RSI = pd.DataFrame()
    RSI['Closing Prices'] = faang_data[ticker]
    RSI['Change'] = RSI['Closing Prices'].diff()
    
    changes = RSI['Change'].tolist()
    
    gains = []
    losses = []
    
    for change in changes:
        if change > 0:
            gains.append(change)
            losses.append(0)
        else:
            gains.append(0)
            losses.append(-change)
    
    gains =  pd.Series(gains)
    losses = pd.Series(losses)
    
    RSI['Gains'] = gains.values
    RSI['Losses'] = losses.values
    
    RSI['Avg. Gain'] = RSI['Gains'].rolling(window=14).mean()
    RSI['Avg. Loss'] = RSI['Losses'].rolling(window=14).mean()
    
    RSI['RS'] = RSI['Avg. Gain'] / RSI['Avg. Loss']
    
    RSI['RSI'] = 100 - 100 / (1 + RSI['RS'])
    
    
    return RSI

In [56]:
# Function to get the MACD crossover

def get_MACD(ticker):
    ShortEMA = get_EMA(ticker, 12)
    LongEMA = get_EMA(ticker, 26)
    
    MACD = pd.DataFrame()
    MACD['12 Day EMA'] = ShortEMA['EMA']
    MACD['26 Day EMA'] = LongEMA['EMA']
    MACD['MACD'] = MACD['12 Day EMA'] - MACD['26 Day EMA']
    
    return MACD